In [ ]:
using StatsBase: mean
using NLopt
using ChainRules
using PyPlot
using LinearAlgebra
using SparseArrays
using Zygote
using DelimitedFiles
using BenchmarkTools

In [ ]:
function Maxwell_2d(Lx, Ly, ϵ, ω, dpml, resolution;
    Rpml=1e-20)

    nx = n_func(Lx, dpml, resolution) #nb point in x
    ny = n_func(Ly, dpml, resolution) #nb points in y
    npml = round(Int, dpml*resolution)
    δ = 1/resolution
    σ0 = -log(Rpml) / (4dpml^3/3)

    # coordinates centered in (0,0)
    x = (1-npml:nx-npml) * δ
    y = (1-npml:ny-npml) * δ
    
    #define the laplacian operator in x direction
    o = ones(nx)/δ
    
    σ = Float64[ξ>Lx ? σ0 * (ξ-Lx)^2 : ξ<0 ? σ0 * ξ^2 : 0.0 for ξ in x]
    Σ = spdiagm(0 => 1.0 ./(1 .+ (im/ω)*σ))
    
    Imat, J, V = SparseArrays.spdiagm_internal(-1 => -o[1:end-1], 0 => o);
    D = sparse(Imat, J, V, nx, nx)
    D[1,end] = -1/δ #periodic boundary condition in x direction
    ∇2x = Σ * transpose(D) * Σ * D

    #define the laplacian operator in y direction
    o = ones(ny) / δ
    y′=((-npml:ny-npml) .+ 0.5) * δ

    σ = Float64[ξ>Ly ? σ0 * (ξ-Ly)^2 : ξ<0 ? σ0 * ξ^2 : 0.0 for ξ in y]
    Σ = spdiagm(0 => 1.0 ./(1 .+ (im/ω)*σ))
    σ′ = Float64[ξ>Ly ? σ0 * (ξ-Ly)^2 : ξ<0 ? σ0 * ξ^2 : 0.0 for ξ in y′]
    Σ′ = spdiagm(0 => 1.0 ./(1 .+ (im/ω)*σ′))
    
    Imat, J, V = SparseArrays.spdiagm_internal(-1 => -o, 0 => o);
    D = sparse(Imat, J, V, ny+1, ny)
    ∇2y = Σ * transpose(D) * Σ′ * D

    #get 2d laplacian using kronecker product
    Ix = sparse(1.0I, nx, nx)
    Iy = sparse(1.0I, ny, ny)
    ∇2d = (kron(Ix, ∇2y) + kron(∇2x, Iy))

    if isa(ϵ, Function)
        geometry = ComplexF64[ϵ(ξ, ζ) for ζ in y, ξ in x]
    else
        geometry = ϵ 
    end

    return (∇2d - spdiagm(0 => reshape(ω^2 * geometry, length(x)*length(y))),
    nx, ny, x, y)
end


# Problem definition

In [ ]:
changevarx(x) = x-2
changevary(y) = y-2

const dpml = 1
const Lx = 4
const Ly = 5
const ω = 2pi
const resolution = 40

const dsource = 1.5

In [ ]:
n_func(Lx, dpml, resolution) = round(Int, (Lx + 2*dpml) * resolution) #nb point in x

function x_func(Ly, dpml, resolution)
    ny = n_func(Ly, dpml, resolution) #nb points in y
    npml = round(Int, dpml*resolution)
    δ = 1/resolution
    return (1-npml:ny-npml) * δ
end

In [ ]:
function create_geom_nopml(ϵ, ny, nx; defaulteps = 0)
    geom = ones((ny, nx))
    if defaulteps!=0
        geom[design_domain, :] .= defaulteps
    end
    geom[design_domain, non_pml_x] = ϵ
    return geom
end

function create_geom(ϵ, ny, nx)
    geom = ones((ny, nx))
    geom[design_domain, :] = ϵ
    return geom
end

function create_target(Lx, Ly, dpml, resolution, target_domain)
    
    nx = n_func(Lx, dpml, resolution)
    ny = n_func(Ly, dpml, resolution)
    target = zeros((ny, nx))
    
    x = x_func(Lx, dpml, resolution)
    y = x_func(Ly, dpml, resolution)
    mask_x = -0.5 .<= changevarx.(x) .<= 0.5
    mask_y = 1 .<= changevary.(y) .<= 2
    
    target[mask_y, mask_x] .= 1
    
    return target[target_domain, non_pml_x]
end

In [ ]:
y = x_func(Ly, dpml, resolution)
x = x_func(Lx, dpml, resolution)

const design_domain = -1 .<= changevary.(y) .<= 0;
const target_domain = 0 .<= changevary.(y) .<= 3;
const non_pml_x = -2 .<= changevary.(x) .<= 2;

const target = create_target(Lx, Ly, dpml, resolution, target_domain); # square [-0.5, 1], [0.5, 2]

In [ ]:
imshow(target)
colorbar()

In [ ]:
const ub = 12
const lb = 1;

# const defaulteps = (lb+ub)/2

In [ ]:
function solve_field(ϵ)
    
    nx = n_func(Lx, dpml, resolution) #nb point in x
    ny = n_func(Ly, dpml, resolution) #nb points in y
#     geom = create_geom_nopml(ϵ, ny, nx, defaulteps=defaulteps)
    geom = create_geom(ϵ, ny, nx)
    
    A, nx, ny, x, y=Maxwell_2d(Lx, Ly, geom, ω, dpml, resolution);

    J = zeros(ComplexF64, (ny, nx))
    J[Integer(dsource * resolution), :]  = 1im  * ω * ones(nx) * resolution #plane source
    Ez = reshape(A \ J[:], (ny, nx));
    return A, Ez, ny, nx, x, y
end

# Optimization problem setup

## define gradient using adjoint method

In [ ]:
"""
`MSE = MSE_func(ϵ)`
this function returns the MSE between the intensity (defined as the abs2 of the electric field) and the target field

ϵ is the permitivity of the design domain
"""
function MSE_func(ϵ)
    A, Ez, ny, nx, x, y = solve_field(ϵ)
    return mean((abs2.(Ez[target_domain, non_pml_x]) .- target).^2)
end

function ChainRules.rrule(::typeof(MSE_func), ϵ)
    A, Ez, ny, nx, x, y = solve_field(ϵ)
    
    target_points = zeros(ComplexF64, (ny, nx))
    target_points[target_domain,non_pml_x] = 2/length(target) .*(abs2.(Ez[target_domain, non_pml_x]) .- target).*conj.(Ez[target_domain, non_pml_x])
#     gradient = 2 * real(ω^2 .*reshape(conj.(A' \ conj.(target_points[:])) .* Ez[:], (ny, nx))[design_domain, non_pml_x]);
    gradient = 2 * real(ω^2 .*reshape(conj.(A' \ conj.(target_points[:])) .* Ez[:], (ny, nx))[design_domain, :]);

        
    function pullback(Δ)
        return (NO_FIELDS, gradient .* Δ)
    end
    
    return mean((abs2.(Ez[target_domain, non_pml_x]) .- target).^2), pullback    
end

Zygote.refresh()

In [ ]:
cur_design = [lb for y_ in y[design_domain], x_ in x]; #initial guess with pml

In [ ]:
MSE_func(cur_design)

In [ ]:
MSE_func'(cur_design)[:]

## inverse design using nlopt

In [ ]:
it = 0
f_evals = []

function myfunc(x::Vector, grad::Vector)
    
    global it, f_evals
    cur_design = reshape(x, (:, n_func(Lx, dpml, resolution)))
    
    if length(grad) > 0
        grad[:] = MSE_func'(cur_design)[:]
    end
    
    f = MSE_func(cur_design)
    println("f_$it = $f")
    it+=1
    push!(f_evals, f)
    return f
end

In [ ]:
cur_design = [lb for y_ in y[design_domain], x_ in x]; #initial guess with pml

opt = Opt(:LD_MMA, length(cur_design))

opt.lower_bounds = lb .* ones(length(cur_design))
opt.upper_bounds = ub .* ones(length(cur_design))
opt.xtol_rel = 1e-8

opt.min_objective = myfunc

@time (minf,minx,ret) = optimize(opt, cur_design[:])
numevals = opt.numevals # the number of function evaluations
println("got $minf after $numevals iterations (returned $ret)")

In [ ]:
A, Ez, ny, nx, x, y = solve_field(minx);

In [ ]:
loglog(f_evals)
ylabel("MSE")
xlabel("# optimization step")

In [ ]:
optimal_design = reshape(minx, (:, n_func(Lx, dpml, resolution)))

writedlm("optimal_design_eps1_.csv", optimal_design, ',')

# Generating figures

In [ ]:
optimal_design = readdlm("optimal_design_eps1.csv", ',')

A, Ez, ny, nx, x, y = solve_field(optimal_design);

In [ ]:
func = abs2
cmapname = "viridis"

subplot(1,2,1)
title("$func(Ez)")
contourf(changevarx.(x),changevary.(y),func.(Ez), cmap=cmapname, levels=500)
xlabel("x")
ylabel("y")
colorbar()

subplot(1,2,2)
title("geometry")
epsilonvals = create_geom(optimal_design, ny, nx)
sourcecolor = maximum(epsilonvals)
contourf(changevarx.(x),changevary.(y), epsilonvals, 
    cmap="viridis", levels=100)
xlabel("x")
ylabel("y")
colorbar()

tight_layout()

In [ ]:
@btime MSE_func'(optimal_design)

In [ ]:
@btime MSE_func(optimal_design)

In [ ]:
writed("intensity_FDFD.txt", abs2.(Ez), ' ')

In [ ]:
writedlm("epsilonvals_FDFD.txt", epsilonvals, ' ')

In [ ]:
writedlm("x_axis_FDFD.txt", changevarx.(x), ' ')

In [ ]:
writedlm("y_axis_FDFD.txt", changevary.(y), ' ')